<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/guideline/Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformation de base
Les transformations sont au cœur de la capacité de traitement des données de Spark. Lorsque vous appliquez une transformation à un RDD, vous créez un nouveau RDD. Les transformations sont paresseuses, ce qui signifie qu’elles ne calculent pas leurs résultats immédiatement. Au lieu de cela, ils se souviennent simplement des transformations appliquées à un ensemble de données de base (par exemple, un fichier). Les transformations ne sont calculées que lorsqu'une action nécessite qu'un résultat soit renvoyé au programme pilote.


In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("Transformation Notebook")
sc = SparkContext.getOrCreate(conf = conf)

## map
La transformation map applique une fonction à chaque élément du RDD et renvoie un nouveau RDD représentant les résultats


In [3]:
data = sc.parallelize((1, 2, 3, 4))
squaredData = data.map(lambda x: x * x)
squaredData.collect()

[1, 4, 9, 16]

## filter
La filter permet de sélectionner des éléments du RDD qui répondent à certains critères.

In [4]:
data = sc.parallelize((1, 2, 3, 4))
evenData = data.filter(lambda x : x % 2 == 0)
evenData.collect()

[2, 4]

## flatMap
Tout en map une fonction à chaque élément individuellement, flatMap peut produire plusieurs éléments de sortie pour chaque élément d'entrée


In [5]:
wordsList = sc.parallelize(("hello world", "how are you"))
words = wordsList.flatMap(lambda x : x.split(" "))
words.collect()

['hello', 'world', 'how', 'are', 'you']

## union
La union fusionne deux RDD

In [6]:
rdd1 = sc.parallelize((1, 2, 3))
rdd2 = sc.parallelize((4, 5, 6))
commonRDD = rdd1.union(rdd2)
commonRDD.collect()

[1, 2, 3, 4, 5, 6]

## intersection
La intersection renvoie un nouveau RDD qui contient uniquement les éléments trouvés dans les deux RDD sources.


In [7]:
rdd1 = sc.parallelize((1, 2, 3))
rdd2 = sc.parallelize((3, 4, 5))
commonRDD = rdd1.intersection(rdd2)
commonRDD.collect()

[3]

## distinct
distinct supprime les doublons dans un RDD

In [8]:
rdd = sc.parallelize((1, 2, 3, 2, 3))
distinctRDD = rdd.distinct()
distinctRDD.collect()

[1, 2, 3]

## substract
subtract renvoie un RDD avec des éléments d'un RDD qui ne sont pas trouvés dans un autre.

In [9]:
rdd1 = sc.parallelize((1, 2, 3, 4))
rdd2 = sc.parallelize((3, 4, 5, 6))
resultRDD = rdd1.subtract(rdd2)
resultRDD.collect()

[2, 1]

## cartesian
cartesian renvoie toutes les paires possibles de (a, b) où a est dans un RDD et b est dans l'autre.

In [10]:
rdd1 = sc.parallelize((1, 2))
rdd2 = sc.parallelize(("a", "b"))
cartesianRDD = rdd1.cartesian(rdd2)
cartesianRDD.collect()

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]

# Transformations cle-valeur
Lorsque vous travaillez avec des RDD de paires clé-valeur, il existe des transformations supplémentaires qui permettent des agrégations complexes et d'autres opérations sur des paires en fonction de leur clé.


## reduceByKey
reduceByKey agrège les valeurs de chaque clé, à l'aide d'une fonction de réduction associative.

In [11]:
data = sc.parallelize([("apple", 2), ("orange", 3), ("apple", 1)])
reducedData = data.reduceByKey(lambda a, b: a+b)
reducedData.collect()

[('apple', 3), ('orange', 3)]

## groupByKey
groupByKey regroupe les valeurs de chaque clé du RDD en une seule séquence. Notez qu’il reduceByKeys’agit souvent d’un meilleur choix car il présente de meilleures caractéristiques de performance.

In [12]:
data = sc.parallelize([("apple", 2), ("orange", 3), ("apple", 1)])
groupedData = data.groupByKey()
groupedData.collect()

[('apple', <pyspark.resultiterable.ResultIterable at 0x7c6bece87ad0>),
 ('orange', <pyspark.resultiterable.ResultIterable at 0x7c6bece85250>)]

## sortByKey
sortByKey trie le RDD par clé.

In [13]:
data = sc.parallelize([("apple", 2), ("orange", 3), ("banana", 1)])
sortedData = data.sortByKey()
sortedData.collect()

[('apple', 2), ('banana', 1), ('orange', 3)]

## join
join rejoint deux RDD par clé

In [14]:
rdd1 = sc.parallelize([("apple", 5), ("orange", 3)])
rdd2 = sc.parallelize([("apple", 2), ("orange", 4)])
joinedRDD = rdd1.join(rdd2)
joinedRDD.collect()

[('apple', (5, 2)), ('orange', (3, 4))]

## cogroup
cogroup regroupe les valeurs de chaque clé dans deux RDD. Cela peut être utile pour les jointures complexes sur plusieurs RDD

In [18]:
rdd1 = sc.parallelize([("apple", 2), ("orange", 3)])
rdd2 = sc.parallelize([("apple", 4), ("orange", 5)])
cogroupedRDD = rdd1.cogroup(rdd2)
cogroupedRDD.collect()

[('apple',
  (<pyspark.resultiterable.ResultIterable at 0x7c6bece81450>,
   <pyspark.resultiterable.ResultIterable at 0x7c6bfd86f250>)),
 ('orange',
  (<pyspark.resultiterable.ResultIterable at 0x7c6bece61790>,
   <pyspark.resultiterable.ResultIterable at 0x7c6bece23ed0>))]